In [1]:
import pandas as pd

df = pd.read_csv('./boston_train.csv')
df.rename(columns = {'medv' : 'PRICE'}, inplace = True)

target_y = df['PRICE']
data_X = df.drop(['PRICE'], axis=1, inplace=False)

In [2]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
import numpy as np

def get_scaled_data(method='None', p_degree=None, input_data=None):
    if method == 'Standard':
        scaled_data = StandardScaler().fit_transform(input_data)
        
    elif method == 'MinMax':
        scaled_data = MinMaxScaler().fit_transform(input_data)
        
    elif method == 'Log':
        scaled_data = np.log1p(input_data)
        
    else:
        scaled_data = input_data

    if p_degree != None:
        scaled_data = PolynomialFeatures(degree=p_degree, include_bias=False).fit_transform(scaled_data)
    
    return scaled_data

In [3]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import cross_val_score

def get_linear_reg_eval(model_name, params=None, data_X_n=None, target_y_n=None, verbose=True, return_coeff=True):
    coeff_df = pd.DataFrame()
    
    if verbose:
        print('####### ', model_name , '#######')
        
    for param in params:
        if model_name =='Ridge':
            model = Ridge(alpha=param)
        elif model_name =='Lasso':
            model = Lasso(alpha=param)
        elif model_name =='ElasticNet':
            model = ElasticNet(alpha=param, l1_ratio=0.7)

        neg_mse_scores = cross_val_score(model, data_X_n, target_y_n, scoring="neg_mean_squared_error", cv=5)
        avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
        
        print('alpha {0}일 때 5 폴드 세트의 평균 RMSE: {1:.3f} '.format(param, avg_rmse))
        
        model.fit(data_X_n, target_y_n)
        if return_coeff:
            coeff = pd.Series(data=model.coef_, index=data_X_n.columns)
            
            colname='alpha:'+str(param)
            coeff_df[colname] = coeff
    
    return coeff_df

In [4]:
alphas = [0.1, 1, 10, 100]

scale_methods=[
    (None, None),
    ('Standard', None),
    ('Standard', 2),
    ('MinMax', None),
    ('MinMax', 2),
    ('Log', None)]

for scale_method in scale_methods:
    data_X_scaled = get_scaled_data(method=scale_method[0], p_degree=scale_method[1], input_data=data_X)
    
    print(data_X_scaled.shape, data_X.shape)
    print('\n## 변환 유형:{0}, Polynomial Degree:{1}'.format(scale_method[0], scale_method[1]))
    
    get_linear_reg_eval('Ridge', params=alphas, data_X_n=data_X_scaled, target_y_n=target_y, verbose=False, return_coeff=False)

(333, 14) (333, 14)

## 변환 유형:None, Polynomial Degree:None
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 6.700 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 6.613 
alpha 10일 때 5 폴드 세트의 평균 RMSE: 6.472 
alpha 100일 때 5 폴드 세트의 평균 RMSE: 6.207 
(333, 14) (333, 14)

## 변환 유형:Standard, Polynomial Degree:None
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 6.731 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 6.694 
alpha 10일 때 5 폴드 세트의 평균 RMSE: 6.466 
alpha 100일 때 5 폴드 세트의 평균 RMSE: 6.251 
(333, 119) (333, 14)

## 변환 유형:Standard, Polynomial Degree:2
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 13.821 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 8.922 
alpha 10일 때 5 폴드 세트의 평균 RMSE: 5.962 
alpha 100일 때 5 폴드 세트의 평균 RMSE: 5.140 
(333, 14) (333, 14)

## 변환 유형:MinMax, Polynomial Degree:None
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 6.626 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 6.267 
alpha 10일 때 5 폴드 세트의 평균 RMSE: 6.693 
alpha 100일 때 5 폴드 세트의 평균 RMSE: 8.229 
(333, 119) (333, 14)

## 변환 유형:MinMax, Polynomial Degree:2
alpha 0.1일 때 5 폴드 세트의 평균 RMSE: 5.324 
alpha 1일 때 5 폴드 세트의 평균 RMSE: 4.892 
alpha 10일 때 5 폴드 세트의 평균 RM